# Tokusan Tutorial: Japanese-Friendly LIME Explanations

This notebook demonstrates how to use the **tokusan** package to explain text classification models with support for Japanese language.

## Contents

1. [Installation & Setup](#1-installation--setup)
2. [Loading the Dataset](#2-loading-the-dataset)
3. [Building a Text Classifier](#3-building-a-text-classifier)
4. [Creating Explanations with Tokusan](#4-creating-explanations-with-tokusan)
5. [Plain Language Explanations (English)](#5-plain-language-explanations-english)
6. [Plain Language Explanations (Japanese)](#6-plain-language-explanations-japanese)
7. [Visualization](#7-visualization)
8. [Advanced Usage](#8-advanced-usage)

## 1. Installation & Setup

First, let's install the required dependencies.

In [ ]:
# Install tokusan (run from the project root)
# !pip install -e ..

# Install Japanese tokenizer support
# !pip install sudachipy sudachidict_core

# Install additional dependencies for this tutorial
# !pip install pandas scikit-learn matplotlib

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

# Import tokusan
import sys
sys.path.insert(0, '..')

from tokusan import (
    TextExplainer,
    active_japanese_tokenizer,
    japanese_splitter,
    generate_sentence_for_feature,
    generate_sentence_for_feature_jp,
    summarize_lime_explanation,
    summarize_lime_explanation_jp,
    print_lime_narrative,
    print_lime_narrative_jp
)

print("Tokusan imported successfully!")
print(f"Active Japanese tokenizer: {active_japanese_tokenizer()}")

## 2. Loading the Dataset

We'll use a Japanese fake news dataset for this tutorial. The dataset contains news articles labeled as real (0) or fake (2).

In [ ]:
# Load the dataset
df = pd.read_csv('fakenews.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nLabel distribution:")
print(df['isfake'].value_counts())

In [ ]:
# Preview the data
df.head()

In [ ]:
# Filter to binary classification: real (0) vs fake (2)
# We exclude label 1 for cleaner binary classification
df_binary = df[df['isfake'].isin([0, 2])].copy()

# Convert labels: 0 = Real, 1 = Fake
df_binary['label'] = (df_binary['isfake'] == 2).astype(int)

print(f"Binary dataset shape: {df_binary.shape}")
print(f"\nLabel distribution:")
print(df_binary['label'].value_counts())
print("\n0 = Real news, 1 = Fake news")

In [ ]:
# Sample a smaller subset for faster training (optional)
# Use the full dataset if you have time
SAMPLE_SIZE = 2000

if len(df_binary) > SAMPLE_SIZE:
    df_sample = df_binary.sample(n=SAMPLE_SIZE, random_state=42)
else:
    df_sample = df_binary

print(f"Sample size: {len(df_sample)}")

## 3. Building a Text Classifier

We'll build a simple text classifier using TF-IDF and Logistic Regression. For Japanese text, we need to use a custom tokenizer.

In [ ]:
# Define a custom tokenizer for Japanese text
def japanese_tokenizer(text):
    """
    Tokenize Japanese text using tokusan's splitter.
    Removes punctuation and short tokens.
    """
    import re
    
    # Get tokens using tokusan's Japanese splitter
    tokens = japanese_splitter(text)
    
    # Filter out punctuation and very short tokens
    punctuation_pattern = re.compile(r'^[\s\.,!?;:"\'\'\"\(\)\[\]\{\}\-\—\–\.\。\、\！\？\「\」\『\』\（\）\・]+$')
    
    filtered_tokens = [
        token for token in tokens 
        if len(token) > 1 and not punctuation_pattern.match(token)
    ]
    
    return filtered_tokens

# Test the tokenizer
test_text = "これは日本語のテストです。機械学習モデルを説明します。"
tokens = japanese_tokenizer(test_text)
print(f"Input: {test_text}")
print(f"Tokens: {tokens}")

In [ ]:
# Split the data
X = df_sample['context'].values
y = df_sample['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

In [ ]:
# Create TF-IDF vectorizer with Japanese tokenizer
vectorizer = TfidfVectorizer(
    tokenizer=japanese_tokenizer,
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2
)

# Build the classifier pipeline
classifier = make_pipeline(
    vectorizer,
    LogisticRegression(max_iter=1000, random_state=42)
)

# Train the model
print("Training Logistic Regression model...")
classifier.fit(X_train, y_train)

# Evaluate
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nModel Accuracy: {accuracy:.2%}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

## 4. Creating Explanations with Tokusan

Now let's use tokusan to explain individual predictions. The key class is `TextExplainer`.

In [ ]:
# Create the explainer for Japanese text
explainer = TextExplainer(
    class_names=['Real', 'Fake'],  # Class names for display
    lang='jp',                      # Enable Japanese tokenization
    random_state=42                 # For reproducibility
)

print("TextExplainer created with Japanese language support.")
print(f"Class names: {explainer.class_names}")

In [ ]:
# Select a sample to explain
sample_idx = 0
sample_text = X_test[sample_idx]
true_label = y_test[sample_idx]

print(f"Sample text (first 200 chars): {sample_text[:200]}...")
print(f"\nTrue label: {'Fake' if true_label == 1 else 'Real'}")

# Get model prediction
pred_proba = classifier.predict_proba([sample_text])[0]
print(f"\nModel prediction:")
print(f"  Real: {pred_proba[0]:.3f}")
print(f"  Fake: {pred_proba[1]:.3f}")

In [ ]:
# Generate explanation
print("Generating LIME explanation...")

explanation = explainer.explain_instance(
    sample_text,
    classifier.predict_proba,
    num_features=10,      # Number of top features to include
    num_samples=1000,     # Number of perturbations (more = more accurate)
    top_labels=2          # Explain top 2 labels
)

print("Explanation generated!")
print(f"\nPrediction probabilities: {explanation.predict_proba}")

In [ ]:
# Get explanation as a list of (word, weight) tuples
print("=" * 60)
print("Explanation for 'Fake' class (label=1):")
print("=" * 60)

fake_explanation = explanation.as_list(label=1)
for word, weight in fake_explanation:
    direction = "↑" if weight > 0 else "↓"
    print(f"  {direction} {word}: {weight:+.4f}")

In [ ]:
# Get explanation for 'Real' class
print("=" * 60)
print("Explanation for 'Real' class (label=0):")
print("=" * 60)

real_explanation = explanation.as_list(label=0)
for word, weight in real_explanation:
    direction = "↑" if weight > 0 else "↓"
    print(f"  {direction} {word}: {weight:+.4f}")

## 5. Plain Language Explanations (English)

Tokusan can generate natural language explanations that are easier to understand than raw weights.

In [ ]:
# Generate English sentence for individual features
print("=" * 60)
print("Individual Feature Explanations (English):")
print("=" * 60)

for word, weight in fake_explanation[:5]:
    sentence = generate_sentence_for_feature(word, weight, "Fake")
    print(f"\n• {sentence}")

In [ ]:
# Generate full English summary
print("=" * 60)
print("Full English Summary:")
print("=" * 60)

english_summary = summarize_lime_explanation(explanation, class_idx=1)
for sentence in english_summary:
    print(f"\n• {sentence}")

In [ ]:
# Print formatted narrative
print_lime_narrative(explanation, class_idx=1)

## 6. Plain Language Explanations (Japanese)

Tokusan's key feature is generating explanations in natural Japanese language.

In [ ]:
# Generate Japanese sentence for individual features
print("=" * 60)
print("個々の特徴量の説明（日本語）:")
print("=" * 60)

for word, weight in fake_explanation[:5]:
    sentence = generate_sentence_for_feature_jp(word, weight, "フェイク")
    print(f"\n・{sentence}")

In [ ]:
# Generate Japanese summary
# Note: The Japanese summary function uses class names from the explanation object
print("=" * 60)
print("日本語による総合説明:")
print("=" * 60)

japanese_summary = summarize_lime_explanation_jp(explanation, class_idx=1)
for sentence in japanese_summary:
    print(f"\n・{sentence}")

In [ ]:
# Print formatted Japanese narrative
print_lime_narrative_jp(explanation, class_idx=1)

## 7. Visualization

Tokusan can create visualizations of feature importances.

In [ ]:
# Create matplotlib bar chart for 'Fake' class
fig = explanation.as_pyplot_figure(label=1, figsize=(10, 6))
plt.tight_layout()
plt.show()

In [ ]:
# Create matplotlib bar chart for 'Real' class
fig = explanation.as_pyplot_figure(label=0, figsize=(10, 6))
plt.tight_layout()
plt.show()

In [ ]:
# Custom visualization comparing both classes
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for idx, (label, label_name) in enumerate([(0, 'Real'), (1, 'Fake')]):
    exp_list = explanation.as_list(label=label)
    
    words = [w for w, _ in exp_list]
    weights = [wt for _, wt in exp_list]
    
    colors = ['green' if w > 0 else 'red' for w in weights]
    
    axes[idx].barh(range(len(words)), weights, color=colors)
    axes[idx].set_yticks(range(len(words)))
    axes[idx].set_yticklabels(words)
    axes[idx].set_xlabel('Weight')
    axes[idx].set_title(f'Feature Importance for "{label_name}"')
    axes[idx].axvline(x=0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

## 8. Advanced Usage

Let's explore some advanced features of tokusan.

In [ ]:
# Explain multiple samples
print("Explaining multiple samples...")
print("=" * 60)

for i in range(3):
    text = X_test[i]
    true = y_test[i]
    pred = classifier.predict([text])[0]
    
    exp = explainer.explain_instance(
        text,
        classifier.predict_proba,
        num_features=5,
        num_samples=500
    )
    
    print(f"\nSample {i+1}:")
    print(f"  Text: {text[:100]}...")
    print(f"  True: {'Fake' if true == 1 else 'Real'}, Predicted: {'Fake' if pred == 1 else 'Real'}")
    print(f"  Top words for prediction:")
    
    for word, weight in exp.as_list(label=pred)[:3]:
        print(f"    - {word}: {weight:+.4f}")

In [ ]:
# Using plain text explanation method
sample_text = X_test[0]

exp = explainer.explain_instance(
    sample_text,
    classifier.predict_proba,
    num_features=5,
    num_samples=500
)

# Get simple plain text summary
plain_summary = explainer.explain_instance_plain_text(exp, label=1, n_words=3)
print("Plain text summary:")
print(plain_summary)

In [ ]:
# Character-level explanation (useful for some models)
char_explainer = TextExplainer(
    class_names=['Real', 'Fake'],
    char_level=True,  # Enable character-level analysis
    random_state=42
)

# Use a short text for character-level
short_text = X_test[0][:50]
print(f"Analyzing text at character level: {short_text}")

char_exp = char_explainer.explain_instance(
    short_text,
    classifier.predict_proba,
    num_features=10,
    num_samples=200
)

print("\nMost important characters:")
for char, weight in char_exp.as_list(label=1)[:5]:
    print(f"  '{char}': {weight:+.4f}")

In [ ]:
# Save explanation to HTML file
exp = explainer.explain_instance(
    X_test[0],
    classifier.predict_proba,
    num_features=10,
    num_samples=500,
    top_labels=2
)

# Save to file
exp.save_to_file('explanation_output.html')
print("Explanation saved to 'explanation_output.html'")

## Summary

In this tutorial, we demonstrated:

1. **Setting up tokusan** with Japanese language support
2. **Building a text classifier** for Japanese fake news detection
3. **Generating LIME explanations** using `TextExplainer`
4. **Plain language explanations** in both English and Japanese
5. **Visualization** of feature importances
6. **Advanced features** like character-level analysis and HTML export

### Key Functions Reference:

| Function | Description |
|----------|-------------|
| `TextExplainer(lang='jp')` | Create explainer with Japanese support |
| `explainer.explain_instance()` | Generate explanation for a text |
| `explanation.as_list(label)` | Get (word, weight) tuples |
| `explanation.as_pyplot_figure()` | Create matplotlib visualization |
| `generate_sentence_for_feature()` | English sentence for one feature |
| `generate_sentence_for_feature_jp()` | Japanese sentence for one feature |
| `summarize_lime_explanation()` | Full English summary |
| `summarize_lime_explanation_jp()` | Full Japanese summary |
| `print_lime_narrative()` | Print formatted English narrative |
| `print_lime_narrative_jp()` | Print formatted Japanese narrative |